# get reviews

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd /content/drive/MyDrive/Project/letterboxd

/content/drive/MyDrive/Project/letterboxd


In [3]:
import requests, time, json, re
import numpy as np
import pandas as pd
from tqdm import tqdm
from datetime import datetime
from bs4 import BeautifulSoup
# Disable chardet debug messages
import logging
logging.getLogger().setLevel(logging.WARNING)
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

from metadata import *

import multiprocessing
cpu_count = multiprocessing.cpu_count()

import os
os.getcwd()

'/content/drive/MyDrive/Project/letterboxd'

In [ ]:
!pip install pandarallel -qq

In [ ]:
from pandarallel import pandarallel
print(cpu_count)
pandarallel.initialize(nb_workers=cpu_count-1, progress_bar=True, use_memory_fs=False)

16
INFO: Pandarallel will run on 15 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


## check num of reviews

In [ ]:
def get_all(row):
    film = row.film
    return movieReview(film).get_num_reviews()

### KR

In [ ]:
kr=pd.read_pickle('data/kr_meta.pkl')

In [ ]:
%%time
kr['num_reviews'] = kr.parallel_apply(get_all, axis=1)

Sleep...
Sleep...
CPU times: user 13.4 s, sys: 3.68 s, total: 17.1 s
Wall time: 3min 41s


In [ ]:
kr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11651 entries, 0 to 11650
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   film         11651 non-null  object 
 1   rate         1536 non-null   float64
 2   meta         11651 non-null  object 
 3   num_reviews  11651 non-null  int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 364.2+ KB


In [ ]:
kr.to_pickle('data/kr_meta.pkl')

### CN

In [ ]:
cn=pd.read_pickle('data/cn_meta.pkl')

In [ ]:
%%time
cn['num_reviews'] = cn.parallel_apply(get_all, axis=1)

CPU times: user 31.9 s, sys: 8.57 s, total: 40.4 s
Wall time: 8min 56s


In [ ]:
cn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28304 entries, 0 to 28303
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   film         28304 non-null  object 
 1   rate         1691 non-null   float64
 2   meta         28304 non-null  object 
 3   num_reviews  28304 non-null  int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 884.6+ KB


In [ ]:
cn.to_pickle('data/cn_meta.pkl')

### JP

In [ ]:
jp=pd.read_pickle('data/jp_meta.pkl')

In [ ]:
%%time
jp['num_reviews'] = jp.parallel_apply(get_all, axis=1)

Sleep...
Sleep...
CPU times: user 39.8 s, sys: 11.1 s, total: 50.8 s
Wall time: 10min 19s


In [ ]:
jp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34689 entries, 0 to 34688
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   film         34689 non-null  object 
 1   rate         4005 non-null   float64
 2   meta         34689 non-null  object 
 3   num_reviews  34689 non-null  int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 1.1+ MB


In [ ]:
jp.to_pickle('data/jp_meta.pkl')

## get detailed reviews for non-zero films

In [4]:
kr=pd.read_pickle('data/kr_meta.pkl')
kr.num_reviews.describe()

count     11651.000000
mean        124.154150
std        2658.527771
min           0.000000
25%           0.000000
50%           0.000000
75%           5.000000
max      233868.000000
Name: num_reviews, dtype: float64

In [5]:
kr1 = kr[kr.num_reviews >= 5].reset_index()

In [70]:
def get_all_reviews(row):
    movie = row['film']
    num = row['num_reviews']
    pages = int(num/12+2)
    review_lst = []
    for page in range(1, pages+1):
        reviews_page_lst = get_review(movie, page)
        review_lst.extend(reviews_page_lst)
    return review_lst

In [6]:
#pbar = tqdm(range(0, len(kr1)))
pbar = tqdm(range(0,3))
reviews = {}

for row in pbar:
    pbar.set_description(f'get {row} out of {len(kr1)}')
    movie = kr1.film[row]
    num = kr1.num_reviews[row]
    pages = int(num/12 + 2)

    review_lst = []
    for page in range(1, pages+1):
        reviews_page_lst = reviewLink(movie, page).get_review()
        review_lst.extend(reviews_page_lst)

    reviews[movie] = review_lst

get 2 out of 3104: 100%|██████████| 3/3 [00:07<00:00,  2.54s/it]


In [7]:
reviews[kr1.film[0]]

[{'rating': '6',
  'date': '20 Feb 2018',
  'review_link': '/hobbify/film/crossroads-of-youth/'},
 {'rating': '7',
  'date': '07 Feb 2019',
  'review_link': '/ewan_m/film/crossroads-of-youth/'},
 {'rating': '2',
  'date': '05 Sep 2019',
  'review_link': '/goose13/film/crossroads-of-youth/'},
 {'rating': '6',
  'date': '24 May 2020',
  'review_link': '/ablackie/film/crossroads-of-youth/'},
 {'rating': '7',
  'date': '04 Oct 2021',
  'review_link': '/pb14/film/crossroads-of-youth/'},
 {'rating': '10',
  'date': '',
  'review_link': '/mythhikal/film/crossroads-of-youth/'},
 {'rating': '7',
  'date': '05 Nov 2022',
  'review_link': '/wawaparkinglot/film/crossroads-of-youth/'},
 {'rating': '7',
  'date': '05 Nov 2022',
  'review_link': '/tinuvielyra/film/crossroads-of-youth/'},
 {'rating': '10',
  'date': '05 Nov 2022',
  'review_link': '/melon21/film/crossroads-of-youth/'},
 {'rating': '8',
  'date': '05 Nov 2022',
  'review_link': '/atraveller/film/crossroads-of-youth/'},
 {'rating': '10'

In [8]:
for key in reviews.keys():
    for i in reviews[key]:
        i['content'] = reviewContent(i['review_link']).get_review_content()

In [9]:
reviews[kr1.film[1]]

[{'rating': '3',
  'date': '27 Sep 2014',
  'review_link': '/punq/film/sweet-dream/',
  'content': "The oldest surviving Korean feature film! Only a curiosity though. They clearly hadn't mastered how to make a good movie yet. All it does is pour in misery until death ends it...."},
 {'rating': '6',
  'date': '02 Jun 2015',
  'review_link': '/mr_vengeance/film/sweet-dream/',
  'content': "Yang Ju-nam's Sweet Dream, from the year 1936, is the oldest surviving Korean film in existence. There have yet to be any older films from the region discovered. This is the sole reason I decided to check this film out as the 1st of my 6 part Classic Korean Cinema Marathon. I went in with essentially no expectations, apart from seeing, again, the oldest surviving Korean film. I walked away not learning too much, nor was the film the greatest, but still, it was solid and very much watchable, even if only in the academic sense. \r\n\r\nSweet Dream is a simply told story of a bickering husband and wife, i